In [1]:
import torch
from Seg import *
from utils import *

In [2]:
SEED = 124
seed_worker, g = set_seed(SEED, desterministic_algorithm=False)

In [3]:
from functools import partial

EXP_NAME = 'kvasir-endo'
IMG_SIZE = 224
TRAIN_BS = 4*7
LR = 1e-2
N_EPOCHS = 500
ALPHA = 1.0
ALPHA_SCHEDULE = True
EMB_SIZE = 256
BETA = 0.00

OPT_FUNC = partial(torch.optim.SGD, lr=LR, momentum=0.95, nesterov=True)
LR_SCHEDULER = 'COSINE_LR'


# Loss
MODE = 'supervised'
MARGIN = 0.3
MINT_ENT_W = 0.005


METRIC = batch_dice_score

VALIDATE_EVERY = 50
TRAIN_NW = 8

def get_training_augmentation(img_size):
    train_transform = [
        albu.OneOf([
            albu.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        ]),
        albu.HorizontalFlip(p=0.5),
        albu.VerticalFlip(p=0.5),
        albu.ShiftScaleRotate(0.05, 0.05, rotate_limit=90, border_mode=cv2.BORDER_CONSTANT, value=0, p=0.2),
        albu.SmallestMaxSize(img_size),
        albu.CenterCrop(img_size, img_size),
    ]
    return albu.Compose(train_transform)

In [4]:
encoder_args = {
    "pretrained":False,           
}
decoder_args = {}


MODEL = fdaUnet(emb_size=EMB_SIZE, encoder_args=encoder_args)

In [5]:
import pandas as pd

train = pd.read_csv('data/csv/Kvasir_SEG/train.csv')
valid = pd.read_csv('data/csv/EndoTect_2020_Segmentation_Test_Dataset/valid.csv')
test = pd.read_csv('data/csv/EndoTect_2020_Segmentation_Test_Dataset/test.csv')

data_path = 'data/images/kvasir/'
train['image'] = data_path + 'image' + train['image']
train['mask'] = data_path + 'mask' + train['mask']

train_ds = fdaSegmentationDataset(train, valid, get_training_augmentation, IMG_SIZE)
valid_ds = SegmentationDataset(test, get_validation_augmentation, IMG_SIZE, normalize=False)
test_ds = SegmentationDataset(test, get_validation_augmentation, IMG_SIZE, normalize=False)

In [ ]:
valid_ds.show_img()

In [ ]:
train_ds.show_img()

In [8]:
from torch.utils.data import DataLoader
train_dl = DataLoader(train_ds, batch_size=TRAIN_BS, 
                      shuffle=True, num_workers=TRAIN_NW, pin_memory=True,
                        worker_init_fn=seed_worker,
                        generator=g, drop_last=True)
valid_dl = DataLoader(valid_ds, batch_size=16, 
                      num_workers=4, shuffle=False,
                     worker_init_fn=seed_worker,
                        generator=g, )

In [9]:
learn = fdaLearner(model=MODEL, data=(train_dl, valid_dl),
                    loss=dice_loss, 
                    name=EXP_NAME, metric=METRIC)

In [10]:
import torch
n_epochs = 1000
optim = OPT_FUNC(learn.model.parameters())

COSINE_LR = partial(torch.optim.lr_scheduler.CosineAnnealingLR, T_max=len(learn.data[0])*N_EPOCHS)
lr_scheduler = eval(LR_SCHEDULER)(optim)

In [11]:
fda_loss = fda_loss(mode=MODE, mint_ent_w=MINT_ENT_W, margin=MARGIN)

In [ ]:
learn.fit(1000, optim, lr_scheduler,
          fda_loss=fda_loss,
          beta=0.01,
          contrastive=True,
          alpha=1.0, alpha_content=1.0,
          schedule_alpha=True,
          scheduler_mode='iteration', validate_every=10)#VALIDATE_EVERY)

In [ ]:
test = pd.read_csv('data/csv/EndoTect_2020_Segmentation_Test_Dataset/test.csv')
test_ds = SegmentationDataset(test, get_validation_augmentation, IMG_SIZE, normalize=False)
test_loader = DataLoader(test_ds, 4, num_workers=4)


learn.get_dice_score(test_loader)